In [ ]:
# Cryptocurrency Analysis Script v002

import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2014, 1, 1).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/indicators/investor_capitalization',
    'https://api.glassnode.com/v1/metrics/supply/current',
    'https://api.glassnode.com/v1/metrics/indicators/liveliness',
    'https://api.glassnode.com/v1/metrics/indicators/realized_profit_lth_account_based',
    'https://api.glassnode.com/v1/metrics/indicators/realized_loss_lth_account_based',
    'https://api.glassnode.com/v1/metrics/supply/profit_relative'
]

def fetch_glassnode_data(url, asset='BTC'):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'f': 'CSV',
        'c': 'USD'
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        df = pd.read_csv(io.StringIO(response.text))
        metric_name = url.split('/')[-1]
        df.columns = ['t', metric_name]
        df['t'] = pd.to_datetime(df['t'], unit='s')
        df[metric_name] = pd.to_numeric(df[metric_name], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]
for metric_url in METRICS:
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

In [18]:

def calculate_supplyinprofit_percentile(df, column='profit_relative', perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate percentile
    percentile = df[column].rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [19]:

def calculate_aviv_percentile(df, columns=['investor_capitalization', 'current', 'liveliness', 'price_usd_close'], perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate AVIV
    aviv = df[columns[2]] * df[columns[1]] * df[columns[3]] / df[columns[0]]
    
    # Calculate percentile
    percentile = aviv.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [20]:

def calculate_lthrealized_plratio_logpercentile(df, columns=['realized_profit_lth_account_based', 'realized_loss_lth_account_based'], perc_window=1400, norm_window=1400, normalization='maxmin'):
    # Calculate LTH realized P/L ratio
    lth_realized_pl = (df[columns[0]] / df[columns[1]]).rolling(window=7).mean()
    
    # Apply log transformation
    log_lth_realized_pl = np.log(lth_realized_pl)
    
    # Calculate percentile
    percentile = log_lth_realized_pl.rolling(window=perc_window).apply(
        lambda x: pd.Series(x).rank(pct=True).iloc[-1]
    )
    
    if normalization == False:
        return percentile
    elif normalization == 'maxmin':
        max_val = percentile.rolling(window=norm_window).max()
        min_val = percentile.rolling(window=norm_window).min()
        normalized_percentile = 2 * (percentile - min_val) / (max_val - min_val) - 1
    elif normalization == 'z-score':
        mean = percentile.rolling(window=norm_window).mean()
        std = percentile.rolling(window=norm_window).std()
        z_score = (percentile - mean) / std
        # Scale z-score to -1 to +1 range
        normalized_percentile = z_score / z_score.abs().max()
    else:
        raise ValueError("Invalid normalization method. Choose 'False', 'maxmin', or 'z-score'.")
    
    return normalized_percentile

In [21]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

def aggregate_indicators(df, columns, method='equal_weight'):
    """
    Aggregate financial behavior indicators from a DataFrame, excluding initial rows with NaN values.
    
    :param df: pandas DataFrame containing the indicators
    :param columns: list of column names to be aggregated
    :param method: 'equal_weight' or 'PCA'
    :return: pandas Series with the aggregated indicator
    """
    # Ensure all specified columns exist in the DataFrame
    if not all(col in df.columns for col in columns):
        raise ValueError("Some specified columns are not in the DataFrame")
    
    # Extract the relevant columns
    data = df[columns]
    
    if method == 'equal_weight':
        # For equal weight, we can use pandas mean which automatically skips NaN
        weight = 1 / len(columns)
        print(f"weights are all equal = {weight:.4f}")
        return data.mean(axis=1)
    
    elif method == 'PCA':
        # Remove rows with any NaN values
        data_clean = data.dropna()
        
        if len(data_clean) == 0:
            raise ValueError("No complete rows found after removing NaN values")
        
        # Standardize the data
        scaler = StandardScaler()
        data_scaled = scaler.fit_transform(data_clean)
        
        # Perform PCA
        pca = PCA(n_components=1)
        pca_result = pca.fit_transform(data_scaled)
        
        # Calculate weights from the first principal component
        weights = pca.components_[0] / np.sum(np.abs(pca.components_[0]))
        
        print("PCA weights:")
        for col, weight in zip(columns, weights):
            print(f"{col}: {weight:.4f}")
        
        # Calculate weighted sum for all rows, including those with NaN
        weighted_sum = data.mul(weights).sum(axis=1)
        
        # Normalize the weighted sum to be in the same range as input data
        min_val, max_val = data.min().min(), data.max().max()
        normalized_sum = (weighted_sum - weighted_sum.min()) / (weighted_sum.max() - weighted_sum.min())
        normalized_sum = normalized_sum * (max_val - min_val) + min_val
        
        return normalized_sum
    
    else:
        raise ValueError("Invalid method. Choose 'equal_weight' or 'PCA'")

In [22]:

# Apply the functions to the merged dataframe with maxmin normalization
merged_df['supplyinprofit_percentile_maxmin'] = calculate_supplyinprofit_percentile(merged_df, normalization='maxmin')
merged_df['aviv_percentile_maxmin'] = calculate_aviv_percentile(merged_df, normalization='maxmin')
merged_df['lthrealized_plratio_logpercentile_maxmin'] = calculate_lthrealized_plratio_logpercentile(merged_df, normalization='maxmin')


In [23]:
merged_df['aggregate_indicator_equalweight']= aggregate_indicators(merged_df, columns=['lthrealized_plratio_logpercentile_maxmin','aviv_percentile_maxmin','supplyinprofit_percentile_maxmin'], method='equal_weight')
merged_df['aggregate_indicator_PCA']= aggregate_indicators(merged_df, columns=['lthrealized_plratio_logpercentile_maxmin','aviv_percentile_maxmin','supplyinprofit_percentile_maxmin'], method='PCA')

weights are all equal = 0.3333
PCA weights:
lthrealized_plratio_logpercentile_maxmin: 0.3325
aviv_percentile_maxmin: 0.3344
supplyinprofit_percentile_maxmin: 0.3331


In [24]:
# Visualization section
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

def create_indicator_chart(merged_df, indicator_column, chart_title):
    # Filter data for the last year
    one_year_ago = datetime.now() - timedelta(days=730)
    merged_df_last_year = merged_df[merged_df.index > one_year_ago]

    # Define a consistent grey color
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

    # Create the visualization
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add price trace
    fig.add_trace(
        go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
        secondary_y=False,
    )

    # Add indicator trace
    indicator = merged_df_last_year[indicator_column]
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator,
            name=indicator_column,
            line=dict(color='green', width=2),
            fill='tozeroy',
            fillcolor='rgba(0,255,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add red color for negative values
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator < 0, 0),
            name=f"{indicator_column} (Negative)",
            line=dict(color='red', width=2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0.1)',
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add y=0 line on top (without adding to legend)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=[0] * len(merged_df_last_year),
            showlegend=False,
            line=dict(color=GREY_COLOR, width=2),
            hoverinfo='skip'
        ),
        secondary_y=True,
    )

    # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
    for month in [1, 4, 7, 10]:
        for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
            date = pd.Timestamp(year=year, month=month, day=1)
            if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
                fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

    # Get the last value of the indicator
    last_value = indicator.iloc[-1]
    last_date = indicator.index[-1]

    # Determine the color based on the last value
    indicator_color = 'green' if last_value >= 0 else 'red'

    # Add annotation for the last value
    fig.add_annotation(
        x=0.95,  # Place at the right edge of the chart
        y=last_value*0.88,  # Place at the vertical position of the last value
        xref="paper",
        yref="y2",  # Use the secondary y-axis for reference
        text=f"{last_value:.2f}",
        showarrow=False,
        font=dict(size=18, color=indicator_color),
        align="left",
        xanchor="left",
        yanchor="middle",
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        showlegend=False,  # Remove legend
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,  # Set width for 16:9 aspect ratio
        height=450,  # Set height for 16:9 aspect ratio
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 14},
        zeroline=False,
        title_text=''  # Remove x-axis title
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=True, 
        range=[-1, 1], 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        side='right',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )

    return fig

In [25]:
def create_aggregator_chart(merged_df, indicator_column, chart_title):
    # Filter data for the last year
    one_year_ago = datetime.now() - timedelta(days=730)
    merged_df_last_year = merged_df[merged_df.index > one_year_ago]

    # Define colors
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
    RED_COLOR = 'red'
    BLUE_COLOR = 'blue'

    # Create the visualization
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add price trace
    fig.add_trace(
        go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
        secondary_y=False,
    )

    # Add indicator trace
    indicator = merged_df_last_year[indicator_column]
    
    # Add red color for positive values with transparent fill
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator > 0, 0),
            name=f"{indicator_column} (Positive)",
            line=dict(color=RED_COLOR, width=2),
            fill='tozeroy',
            fillcolor='rgba(255,0,0,0)',  # Transparent red
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add blue color for negative values with transparent fill
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=indicator.where(indicator < 0, 0),
            name=f"{indicator_column} (Negative)",
            line=dict(color=BLUE_COLOR, width=2),
            fill='tozeroy',
            fillcolor='rgba(0,0,255,0)',  # Transparent blue
            mode='lines'
        ),
        secondary_y=True,
    )

    # Add y=0 line on top (without adding to legend)
    fig.add_trace(
        go.Scatter(
            x=merged_df_last_year.index,
            y=[0] * len(merged_df_last_year),
            showlegend=False,
            line=dict(color=GREY_COLOR, width=2),
            hoverinfo='skip'
        ),
        secondary_y=True,
    )

    # Add vertical lines for every three months (Jan, Mar, May, Jul, Sep, Nov)
    for month in [1, 4, 7, 10]:
        for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
            date = pd.Timestamp(year=year, month=month, day=1)
            if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
                fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

    # Get the last value of the indicator
    last_value = indicator.iloc[-1]

    # Determine the color based on the last value
    indicator_color = RED_COLOR if last_value >= 0 else BLUE_COLOR

    # Add annotation for the last value
    fig.add_annotation(
        x=0.95,  # Place at the right edge of the chart
        y=last_value*0.88,  # Place at the vertical position of the last value
        xref="paper",
        yref="y2",  # Use the secondary y-axis for reference
        text=f"{last_value:.2f}",
        showarrow=False,
        font=dict(size=18, color=indicator_color),
        align="left",
        xanchor="left",
        yanchor="middle",
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        showlegend=False,  # Remove legend
        hovermode="x unified",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,  # Set width for 16:9 aspect ratio
        height=450,  # Set height for 16:9 aspect ratio
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 14},
        zeroline=False
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )
    fig.update_yaxes(
        showgrid=False, 
        secondary_y=True, 
        range=[-1, 1], 
        tickfont={'color': GREY_COLOR, 'size': 14},
        zeroline=False,
        showline=True,
        linecolor=GREY_COLOR,
        ticks='outside',
        side='right',
        tickcolor=GREY_COLOR,
        title_text='',
        title_font=dict(size=18)
    )

    return fig

In [26]:
# Create charts for each indicator
indicators = [
    ('supplyinprofit_percentile_maxmin', "Bitcoin: Supply in Profit"),
    ('aviv_percentile_maxmin', "Bitcoin: AVIV"),
    ('lthrealized_plratio_logpercentile_maxmin', "Bitcoin: LTH Realized P/L Ratio")
]

for indicator, title in indicators:
    fig = create_indicator_chart(merged_df, indicator, title)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [27]:
# Create charts for each aggregator
aggregators = [
    ('aggregate_indicator_equalweight', "Bitcoin: LTH Aggregate Indicator EW"),
    ('aggregate_indicator_PCA', "Bitcoin: LTH Aggregate Indicator PCA")
]

for aggregator, title in aggregators:
    fig = create_aggregator_chart(merged_df, aggregator, title)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_lth_{aggregator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [28]:
merged_df.tail()

,price_usd_close,investor_capitalization,current,liveliness,realized_profit_lth_account_based,realized_loss_lth_account_based,profit_relative,supplyinprofit_percentile_maxmin,aviv_percentile_maxmin,lthrealized_plratio_logpercentile_maxmin,aggregate_indicator_equalweight,aggregate_indicator_PCA
t,,,,,,,,,,,,
2024-10-01,60895.040343,5.655606e+11,1.204851e+12,0.608790,2.107309e+08,3.641067e+06,0.787309,0.154179,0.397425,0.372493,0.308032,0.332472
2024-10-02,60649.186895,5.653674e+11,1.198555e+12,0.608708,1.314906e+08,3.348256e+07,0.779070,0.121037,0.380258,0.342407,0.281234,0.304798
2024-10-03,60740.512269,5.652320e+11,1.201173e+12,0.608644,2.101564e+08,7.740661e+06,0.785471,0.145533,0.383691,0.316619,0.281948,0.305554
2024-10-04,62104.674280,5.655939e+11,1.226021e+12,0.608584,2.457217e+08,2.527413e+06,0.817514,0.249280,0.445494,0.292264,0.329012,0.354246
2024-10-05,62075.343850,5.657100e+11,1.226831e+12,0.608469,7.458537e+07,7.687506e+05,0.816707,0.244957,0.443777,0.283668,0.324134,0.349211


In [29]:
merged_df.to_csv('bitcoin_analysis_data_lth.csv', index=True)

In [30]:
def chart_aggregate(df, indicator_col, price_col, chart_title="Price and Indicator Chart"):
    # Filter for the last 2 years of data
    last_2_years = df.index.max() - pd.DateOffset(years=2)
    df_last_2_years = df[df.index >= last_2_years]

    # Define a consistent grey color
    GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey

    # Create the scatter plot
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=df_last_2_years.index,
            y=df_last_2_years[price_col],
            mode='markers',
            marker=dict(
                color=df_last_2_years[indicator_col],
                size=8,
                line=dict(width=0.5, color='black'),  # Thinner black border for dots
                colorscale='RdYlGn_r',  # Red-Yellow-Green colorscale reversed
                colorbar=dict(
                    title='Indicator',
                    tickvals=[-1, 0, 1],
                    ticktext=['-1', '0', '1'],
                    lenmode='fraction',
                    len=0.9,  # Make the colorbar 90% of the chart height
                    thickness=20,  # Adjust thickness as needed
                    x=1.02,  # Move colorbar to the right side
                    xpad=10,  # Add some padding between colorbar and chart
                ),
                cmin=-1,
                cmax=1,
            ),
            text=[f"Date: {date.strftime('%Y-%m-%d')}<br>Price: {price:.2f}<br>Indicator: {ind:.2f}" 
                  for date, price, ind in zip(df_last_2_years.index, df_last_2_years[price_col], df_last_2_years[indicator_col])],
            hoverinfo='text'
        )
    )

    # Add vertical lines for every 3 months (January, April, July, October)
    start_date = df_last_2_years.index[0].to_period('Q').to_timestamp()
    end_date = df_last_2_years.index[-1].to_period('Q').to_timestamp()
    for date in pd.date_range(start=start_date, end=end_date, freq='Q'):
        if df_last_2_years.index[0] <= date <= df_last_2_years.index[-1]:
            fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=1, opacity=0.7)

    # Get the last value of the indicator and price
    last_value = df_last_2_years[indicator_col].iloc[-1]
    last_price = df_last_2_years[price_col].iloc[-1]

    # Add annotation for the last value
    fig.add_annotation(
        x=df_last_2_years.index[-1],  # Fixed x position at the last date
        y=last_price,  # y position based on the last price
        text=f"{last_value:.2f}",  # Simplified annotation
        showarrow=False,
        font=dict(size=14, color='red' if last_value > 0 else 'green'),  # Color logic based on reversed scale
        align="left",
        xanchor="left",
        yanchor="middle",
        xshift=10,  # Shift the annotation slightly to the right of the last point
    )

    # Update layout
    fig.update_layout(
        title={
            'text': chart_title,
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        showlegend=False,
        hovermode="closest",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=1000,
        height=450,
        margin=dict(l=40, r=120, t=60, b=40)  # Increase right margin to accommodate colorbar
    )

    # Update axes
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': 'black', 'size': 12},
        zeroline=False,
        dtick="M3",  # Show tick every 3 months
        tickformat="%b %Y",  # Format as "Mon Year" for every tick
        showline=False,  # Remove x-axis line
        mirror=False
    )
    fig.update_yaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 12},
        zeroline=False,
        showline=True,
        linewidth=1,
        linecolor=GREY_COLOR,
        mirror=False,
        ticks='outside',  # Show ticks outside
        ticklen=5,  # Length of the ticks
        tickcolor=GREY_COLOR  # Color of the ticks
    )

    return fig

In [31]:

# Example usage
fig2 = chart_aggregate(merged_df, 'aggregate_indicator_equalweight', 'price_usd_close', "LTH Sentiment Aggregated Indicator")
fig2.show()
pio.write_html(fig2, file=f'bitcoin_analysis_lth_aggr_last_2year.html')
print("All charts have been displayed and saved as separate HTML files.")

/var/folders/kw/s9r1_7lx3zn35g56dhqzfmwh0000gn/T/ipykernel_2586/3319785951.py:44: FutureWarning:

'Q' is deprecated and will be removed in a future version, please use 'QE' instead.



All charts have been displayed and saved as separate HTML files.


In [32]:
# Define the color codes (matching the colors in create_indicator_chart)
GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
RED_COLOR = 'red'  # Red color for positive changes
BLUE_COLOR = 'blue'  # Blue color for negative changes

# Function to calculate percentage changes for given time periods
def calculate_periodic_changes(df, column):
    # Calculate the percentage changes for weekly, monthly, and quarterly periods
    weekly_change = df[column].pct_change(7).iloc[-1] * 100  # 7 days for weekly change
    monthly_change = df[column].pct_change(30).iloc[-1] * 100  # 30 days for monthly change
    quarterly_change = df[column].pct_change(90).iloc[-1] * 100  # 90 days for quarterly change

    return weekly_change, monthly_change, quarterly_change

# Create the bar chart for quarterly, monthly, and weekly changes
def create_bar_chart(merged_df, column):
    # Calculate percentage changes for the specified column
    weekly_change, monthly_change, quarterly_change = calculate_periodic_changes(merged_df, column)
    
    # Define the periods and changes in the new order: quarterly, monthly, weekly
    periods = ['Quarterly', 'Monthly', 'Weekly']
    changes = [quarterly_change, monthly_change, weekly_change]
    
    # Determine the color for each bar: RED_COLOR for positive, BLUE_COLOR for negative
    colors = [RED_COLOR if change > 0 else BLUE_COLOR for change in changes]

    # Create the bar chart with the updated bar width
    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=periods,
            y=changes,
            marker_color=colors,  # Set the colors for each bar
            text=[f"{change:.2f}%" for change in changes],  # Add text labels with percentage values
            textposition='outside',
            width=0.5  # Make the bars wider (as per your change)
        )
    )

    # Customize the layout
    fig.update_layout(
        title={
            'text': f"Rate of Change {column}",
            'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
        },
        xaxis_title=None,  # Remove the x-axis title
        yaxis_title="Rate of Change",  # Change "Percentage Change" to "Rate of Change"
        yaxis_ticksuffix="%",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font={'color': 'black', 'size': 14},
        width=600,  # Keep width at 600 pixels
        height=600,  # Keep height at 600 pixels
        hovermode="x unified",
    )

    # Add a horizontal line at y = 0%
    fig.add_hline(y=0, line_dash="solid", line_color=GREY_COLOR, line_width=1)

    # Adjust y-axis tick spacing dynamically based on the fixed height (600)
    y_range = max(changes) - min(changes)  # Calculate the range of y values
    tick_spacing = y_range / 8  # Set a fixed number of ticks (8) based on height

    # Update x-axis (periods)
    fig.update_xaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},  # Make x-axis labels grey
        zeroline=False,
        linecolor=None,  # Remove x-axis line
        showline=False,  # Ensure no line on x-axis
        ticks='outside',
        tickcolor=GREY_COLOR
    )

    # Update y-axis (percentage changes)
    fig.update_yaxes(
        showgrid=False, 
        tickfont={'color': GREY_COLOR, 'size': 14},  # Make y-axis labels grey
        zeroline=False,
        linecolor=GREY_COLOR,  # Grey color for y-axis lines
        showline=True,
        ticks='outside',
        tickcolor=GREY_COLOR,
        dtick=tick_spacing,  # Set dynamic tick spacing
    )

    # Show the chart
    fig.show()

    # Save the chart as an HTML file
    pio.write_html(fig, file=f'{column}_periodic_changes_chart.html')

# Call the function for aggregate indicator columns and save charts
create_bar_chart(merged_df, 'aggregate_indicator_equalweight')
create_bar_chart(merged_df, 'aggregate_indicator_PCA')